In [1]:
import pandas as pd
from datetime import datetime, timedelta
import requests
import json

In [2]:
# By latitude and longitude
# API parameters
options = {}
options["url"] = "https://airnowapi.org/aq/data/"
options["parameters"] = "o3,pm25,pm10,co,no2,so2"
options["bbox"] = "-124.848974, 24.396308, -66.885444, 49.384358"
options["data_type"] = "b"
options["format"] = "application/json"
options["ext"] = "json"
options["api_key"] = "EB883725-96B2-407E-AAD0-7D4FFEF22290"
options['includerawconcentrations'] = "1"

starttime = datetime(2024,2,17,hour=12,minute=15,second=0)

start_str = str(starttime.hour) + ':' + str(starttime.minute)

time_step = 30 # minutes
start_time = "12:00"

for i in range(6):
    endtime = starttime + timedelta(minutes=time_step*i)
    endtime_str = str(endtime.hour) + ':' + str(endtime.minute)
    # API parameters
    options["start_date"] = "2024-02-17"
    options["start_hour_utc"] = start_str
    options["end_date"] = "2024-02-17"
    options["end_hour_utc"] = endtime_str


    # API request URL
    REQUEST_URL = options["url"] \
                  + "?startdate=" + options["start_date"] \
                  + "t" + options["start_hour_utc"] \
                  + "&enddate=" + options["end_date"] \
                  + "t" + options["end_hour_utc"] \
                  + "&parameters=" + options["parameters"] \
                  + "&bbox=" + options["bbox"] \
                  + "&datatype=" + options["data_type"] \
                  + "&format=" + options["format"] \
                  + "&api_key=" + options["api_key"] \
                  + "&includerawconcentrations" + options["includerawconcentrations"]
    
    r = requests.get(REQUEST_URL)
    data = json.loads(r.text)
    df = pd.DataFrame(data)
    print(f'Between {options["start_hour_utc"]} and {options["end_hour_utc"]}, there were {len(df)} observations reported')

Between 12:15 and 12:15, there were 0 observations reported
Between 12:15 and 12:45, there were 0 observations reported
Between 12:15 and 13:15, there were 3251 observations reported
Between 12:15 and 13:45, there were 3251 observations reported
Between 12:15 and 14:15, there were 6507 observations reported
Between 12:15 and 14:45, there were 6507 observations reported


In [3]:
df.head()

,Latitude,Longitude,UTC,Parameter,Unit,Value,AQI,Category
0,49.261014,-124.806628,2024-02-17T13:00,PM2.5,UG/M3,7.1,30,1
1,48.286700,-124.620300,2024-02-17T13:00,OZONE,PPB,37.0,34,1
2,48.286700,-124.620300,2024-02-17T13:00,PM2.5,UG/M3,1.6,7,1
3,48.286700,-124.620300,2024-02-17T13:00,SO2,PPB,0.2,0,1
4,48.286700,-124.620300,2024-02-17T13:00,CO,PPM,0.1,-999,-999


In [73]:
import requests
import json
# Current AQ by lat/long
options = {}
options["url"] = "https://airnowapi.org/aq/observation/latLong/current"
options["latitude"] = "38.33"
options['longitude'] = "-122.28"
options["format"] = "text/csv"
options["api_key"] = "EB883725-96B2-407E-AAD0-7D4FFEF22290"
options["distance"] = "150"

# API request URL
REQUEST_URL = options["url"] \
              + "&latitude=" + options['latitude'] \
              + "&longitude=" + options['longitude'] \
              + "&format=" + options["format"] \
              + "$distance" + options['distance'] \
              + "&api_key=" + options["api_key"]

In [74]:
r = requests.get(REQUEST_URL)

In [75]:
r

<Response [200]>

In [76]:
r.text

'    <!doctype html>\r\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">\r\n    <head>\r\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8"> \r\n    <title>AirNow API Documentation</title>\r\n    <link rel="stylesheet" href="/css/lib/blueprint/screen.css" type="text/css" media="screen, projection">\r\n    <link rel="stylesheet" href="/css/lib/blueprint/print.css" type="text/css" media="print">\r\n    <!--[if lt IE 8]><link rel="stylesheet" href="/css/lib/blueprint/ie.css" type="text/css" media="screen, projection"><![endif]-->\r\n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.1/css/font-awesome.min.css" type="text/css">\r\n    <link rel="stylesheet" href="/css/default.css" type="text/css">\r\n    <script async src="https://www.googletagmanager.com/gtag/js?id=G-8FZ16JDJ5Y"></script>\r\n    <script>\r\n        window.dataLayer = window.dataLayer || [];\r\n        function gtag(){dataLayer.push(arguments);}\r\n    

In [15]:
json.loads(r.text)

JSONDecodeError: Expecting value: line 1 column 5 (char 4)

In [9]:
import requests
import json
# Forecasted AQ by zipcode
options = {}
options["url"] = "https://airnowapi.org/aq/forecast/zipCode/"
options["zipCode"] = '94954'
options['date'] = '2012-02-01'
options["format"] = "application/json"
options["api_key"] = "4d36e978-e325-11cec1-08002be10318"
options["distance"] = "200"

# API request URL
REQUEST_URL = options["url"] \
              + "&zipCode=" + options['zipCode'] \
            + "&date=" + options['date'] \
              + "&format=" + options["format"] \
              + "&api_key=" + options["api_key"]

In [10]:
r = requests.get(REQUEST_URL)
r

<Response [401]>

In [11]:
r.text

'<WebServiceError><Message>Request not authenticated.</Message></WebServiceError>'